In [30]:
!pip3 install transformers  sentence-transformers  qdrant-client underthesea

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 29.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 2.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 5.2 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [58]:
import os
from underthesea import sent_tokenize, text_normalize

file_path = './input_dataset.txt'

def read_file(file_path):
  try:
    with open(file_path, 'r', encoding='utf-8') as file: 
      content = file.read()
      return content
  except FileNotFoundError:
    print(f"Error: File not found at {file_path}")
    return None

file_content = read_file(file_path)
text_chunks = sent_tokenize(text_normalize(file_content))

text_chunks

['Kinh doanhVĩ môThứ tư , 11/12/2024 , 17 : 45 ( GMT + 7 ) Chính phủ đề xuất lập Quỹ Hỗ trợ đầu tư khi áp thuế tối thiểu toàn cầu Chính phủ đề xuất lập Quỹ Hỗ trợ đầu tư với các chính sách mạnh hơn để giữ chân , thu hút các tập đoàn lớn trong lĩnh vực bán dẫn , AI khi áp dụng thuế tối thiểu toàn cầu .',
 'Chiều 11/12 , Ủy ban Thường vụ thảo luận dự thảo Nghị định về thành lập , quản lý và sử dụng Quỹ Hỗ trợ đầu tư .',
 'Thứ trưởng Kế hoạch & Đầu tư Nguyễn Thị Bích Ngọc cho biết hiện thu hút đầu tư của Việt Nam dựa trên chính sách ưu đãi , hỗ trợ đầu tư theo lĩnh vực , địa bàn khuyến khích rót vốn .',
 'Mức ưu đãi cụ thể áp dụng theo quy định về thuế , kế toán và đất đai .',
 'Bà Ngọc cho rằng chính sách như vậy có tính cạnh tranh trong khu vực .',
 'Nhưng chính sách còn dàn trải , chưa bắt kịp với thông lệ quốc tế , đặc biệt không còn tác dụng trong thu hút dự án lớn , tập đoàn đa quốc gia khi áp dụng thuế tối thiểu toàn cầu .',
 'Tức là , Việt Nam cần chính sách hỗ trợ đầu tư mới hướn

In [ ]:
# prompt: get LLM vietnamese model from huggingface for sentence-transformers 

from sentence_transformers import SentenceTransformer, util
import qdrant_client
# Download the pre-trained model
# model = SentenceTransformer('vinai/phobert-base-v2')
model = SentenceTransformer('VoVanPhuc/sup-SimCSE-VietNamese-phobert-base', cache_folder='./llm_cache')

client = qdrant_client.QdrantClient(":memory:")


In [59]:

from qdrant_client.http import models
from qdrant_client.http.models import PointStruct
import uuid

texts = text_chunks

embeddings = model.encode(texts)

client.recreate_collection(
    collection_name="vietnamese_embeddings",
    vectors_config=models.VectorParams(size=embeddings.shape[1], distance=models.Distance.COSINE),
)

points = []
for i, embedding in enumerate(embeddings):
  points.append(PointStruct(id=str(uuid.uuid4()), vector=embedding.tolist(), payload={"text": texts[i]}))

client.upsert(
  collection_name="vietnamese_embeddings",
  wait=True,
  points=points
)



/tmp/ipykernel_1695/1312076986.py:9: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [63]:
# Optional: Search to verify
embeddings = model.encode(['Thông tin cơ bản về HUTECH:'])
search_result = client.search(
    collection_name="vietnamese_embeddings",
    query_vector=embeddings[0].tolist(),
    limit=8
)

# payloads = [e.payload for e in search_result]

search_result 
## Payload nen limit context lenght nen moi phải dung cai khưa nay
## + Payload and system promnt then send to GPT TO GET RESULT.

[ScoredPoint(id='2d9dbe08-e24f-4712-9b09-5282a24530ac', version=0, score=0.5305707351847412, payload={'text': 'Thông tin cơ bản về HUTECH : Năm thành lập : 1995 .'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id='c2b0619d-639d-442c-a8d1-ee97b6850a92', version=0, score=0.5152457883039255, payload={'text': 'Web Backend : Lập trình server với Node .'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id='746f35a0-c700-481f-8f5b-60a49507684d', version=0, score=0.4882955935413674, payload={'text': 'Điều này được thể hiện qua bản thân cái tên – một sự tôn vinh dành cho nhóm hài người Anh Monty Python [ 71 ] – và trong một số cách tiếp cận thi thoảng vui tươi trong hướng dẫn và các tài liệu tham khảo , chẳng hạn như một vi dụ có đề cập đến trứng và spam ( gợi nhắc đến một tiểu phẩm trong Monty Python ) thay cho foo và bar tiêu chuẩn .'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id='99814221-cd96-4adc-ab4d-272bfa2f0f1e', version=0, score=0.4861